In [1]:
"""Swiss Metro Price Changes by Age -- Base case -- youth

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',-0.215775,-20,20,0,'ASC_CAR')
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1,'ASC_SBB')
ASC_SM	 = Beta('ASC_SM',0.122635,-20,20,0,'ASC_SM')
B_CAR_COST	 = Beta('B_CAR_COST',-0.007514,-10,10,0,'B_CAR_COST')
B_HE	 = Beta('B_HE',-0.005887,-10,10,0,'B_HE')
B_SM_COST	 = Beta('B_SM_COST',-0.011591,-10,10,0,'B_SM_COST')
B_CAR_TIME	 = Beta('B_CAR_TIME',-0.013063,-10,10,0,'B_CAR_TIME')
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',-0.008814,-10,10,0,'B_TRAIN_TIME')
B_SM_TIME	 = Beta('B_SM_TIME',-0.010993,-10,10,0,'B_SM_TIME')
B_TRAIN_COST	 = Beta('B_TRAIN_COST',-0.029349,-10,10,0,'B_TRAIN_COST')
B_SENIOR	 = Beta('B_SENIOR',-1.864569,-10,10,0,'B_SENIOR')
B_GA	 = Beta('B_GA',0.518405,-10,80,0,'B_GA')
B_FIRST	 = Beta('B_FIRST',0.261506,-10,80,0,'B_FIRST')

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', 0.8 * SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Exclude
# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 2  ) + (  AGE == 3  ) + (  AGE == 4  ) + (  AGE == 5  ) + (  AGE == 6  ))>0
database.remove(exclude)

#No need for estimating the model (it is already estimated). We want to obtain the individual probabilities and the market shares
prob1 = models.logit(V,av,1)
prob2 = models.logit(V,av,2)
prob3 = models.logit(V,av,3)
#Instead of reporting the choice in the simulation file, the probability of the chosen can be printed

#Simulation output
simulate = {'choice': CHOICE,
            '01 Prob. Train': prob1,
            '02 Prob. SM': prob2,
            '03 Prob. Car': prob3,
            '04 Elast SM_COST Train': Derive(prob1,'SM_COST')*SM_COST/prob1,
            '05 Elast SM_COST SM': Derive(prob2,'SM_COST')*SM_COST/prob2,
            '06 Elast SM_COST Car': Derive(prob3,'SM_COST')*SM_COST/prob3}

biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "MNL_SM_price_base_youth"
results = biogeme.simulate()
print("Results=",results.describe())

07-04-2019:14:14:36,111 WARNING  [biogeme.py:143] The chosen alternative is not available for the following observations (rownumber[choice]): 486[3]-487[3]-488[3]-489[3]-490[3]-491[3]-492[3]-493[3]-494[3]-1989[3]-1990[3]-1991[3]-1992[3]-1993[3]-1994[3]-1995[3]-1996[3]-1997[3]-2025[3]-2026[3]-2027[3]-2028[3]-2029[3]-2030[3]-2031[3]-2032[3]-2033[3]-2034[3]-2035[3]-2036[3]-2037[3]-2038[3]-2039[3]-2040[3]-2041[3]-2042[3]-2043[3]-2044[3]-2045[3]-2046[3]-2047[3]-2048[3]-2049[3]-2050[3]-2051[3]-2052[3]-2053[3]-2054[3]-2055[3]-2056[3]-2057[3]-2058[3]-2059[3]-2060[3]-2241[3]-2242[3]-2243[3]-2244[3]-2245[3]-2246[3]-2247[3]-2248[3]-2249[3]-2349[3]-2350[3]-2351[3]-2352[3]-2353[3]-2354[3]-2355[3]-2356[3]-2357[3]-2358[3]-2359[3]-2360[3]-2361[3]-2362[3]-2363[3]-2364[3]-2365[3]-2366[3]-2394[3]-2395[3]-2396[3]-2397[3]-2398[3]-2399[3]-2400[3]-2401[3]-2402[3]-2403[3]-2404[3]-2405[3]-2406[3]-2407[3]-2408[3]-2409[3]-2410[3]-2411[3]-2421[3]-2422[3]-2423[3]-2424[3]-2425[3]-2426[3]-2427[3]-2428[3]-2429[3]-246

Results=            choice  01 Prob. Train  02 Prob. SM  03 Prob. Car  \
count  423.000000      423.000000   423.000000    423.000000   
mean     1.626478        0.212870     0.719859      0.067271   
std      0.544214        0.126800     0.162194      0.152162   
min      1.000000        0.001373     0.130222      0.000000   
25%      1.000000        0.102915     0.600008      0.000000   
50%      2.000000        0.229194     0.723230      0.000000   
75%      2.000000        0.300262     0.838940      0.030670   
max      3.000000        0.504039     0.996187      0.861078   

       04 Elast SM_COST Train  05 Elast SM_COST SM  06 Elast SM_COST Car  
count              423.000000           423.000000            423.000000  
mean                 0.381383            -0.104529              0.084099  
std                  0.556113             0.282703              0.242763  
min                  0.000000            -2.306671              0.000000  
25%                  0.000000          